In [2]:
import os 
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
def download_dataset():
    '''Downloads a dataset from kaggle and only keeps the csv in your data file. Beware of your own data structure:
    this creates a data directory and also moves all the .csv files next to your jupyter notebooks to it.
    Takes: url from kaggle
    Returns: a folder with the downloaded csv
    '''
    
    #Gets the name of the dataset.zip
    url = input("Introduce la url: ")
    
    #Gets the name of the dataset.zip
    endopint = url.split("/")[-1]
    user = url.split("/")[-2]
    
    #Download, decompress and leaves only the csv
    download = f"kaggle datasets download -d {user}/{endopint}; say -v Monica 'descargando'"
    decompress = f"tar -xzvf {endopint}.zip; say -v Monica 'descomprimiendo'"
    delete = f"rm -rf {endopint}.zip; say -v Monica 'borrando el zip'"
    make_directory = "mkdir data"
    lista = "ls >> archivos.txt"
    
    for i in [download, decompress, delete, make_directory, lista]:
        os.system(i)
    
    #Gets the name of the csv (you should only have one csv when running this code)
    lista_archivos = open('archivos.txt').read()
    nueva = lista_archivos.split("\n")
    
    #Moves the .csv into the data directory
    for i in nueva:
        if i.endswith(".csv"):
            move_and_delete = f"mv {i} nba_data/dataset.csv; rm archivos.txt; say -v Monica 'moviendo el dataset'"
            return os.system(move_and_delete)

In [3]:
download_dataset()

Introduce la url: https://api-nba-v1.p.rapidapi.com/seasons/


In [4]:
data = pd.read_csv("nba_data/dataset.csv", encoding = "ISO-8859-1")

As we will see there are no missing values in the dataset, so we are going to the court just renaming some columns and dropping one of them, the classic one "Unnamed:0". 

In [5]:
data = data.dropna()

In [6]:
data.drop(["Unnamed: 0"], axis = 1, inplace = True)

In [7]:
data.head()

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,Dennis Rodman,CHI,36.0,198.12,99.790240,Southeastern Oklahoma State,USA,1986,2,27,...,5.7,16.1,3.1,16.1,0.186,0.323,0.100,0.479,0.113,1996-97
1,Dwayne Schintzius,LAC,28.0,215.90,117.933920,Florida,USA,1990,1,24,...,2.3,1.5,0.3,12.3,0.078,0.151,0.175,0.430,0.048,1996-97
2,Earl Cureton,TOR,39.0,205.74,95.254320,Detroit Mercy,USA,1979,3,58,...,0.8,1.0,0.4,-2.1,0.105,0.102,0.103,0.376,0.148,1996-97
3,Ed O'Bannon,DAL,24.0,203.20,100.697424,UCLA,USA,1995,1,9,...,3.7,2.3,0.6,-8.7,0.060,0.149,0.167,0.399,0.077,1996-97
4,Ed Pinckney,MIA,34.0,205.74,108.862080,Villanova,USA,1985,1,10,...,2.4,2.4,0.2,-11.2,0.109,0.179,0.127,0.611,0.040,1996-97


In [8]:
data = data.rename(columns={"Full Name":"player_name", "Team": "team_abbreviation", "Age": "age",
                            "Height":"player_height", "Weight": "player_weight", "College":"college",
                            "Country": "country", "Draft Year":"draft_year", "Draft Round": "draft_round",
                            "Draft Number": "draft_number", "Games Played": "gp", "Pts per game": "pts",
                            "Reb per game": "reb", "Assists per game": "ast", "Net Rating": "net_rating",
                            "Off Reb%": "oreb_pct", "Def Reb%": "dreb_pct", "Usage%": "usg_pct", "TS%": "ts_pct",
                            "Assists%": "ast_pct", "Season": "season"
                            })

In [19]:
data.tail() #Have to solve this problem.

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
11140,Maxi Kleber,DAL,28.0,208.28,108.862080,None,Germany,Undrafted,Undrafted,Undrafted,...,9.1,5.4,1.1,4.6,0.056,0.140,0.136,0.605,0.064,2019-20
11141,Melvin Frazier Jr.,ORL,23.0,195.58,97.522280,Tulane,USA,2018,2,35,...,1.2,0.3,0.1,-2.4,0.018,0.058,0.164,0.480,0.033,2019-20
11142,Meyers Leonard,MIA,28.0,213.36,117.933920,Illinois,USA,2012,1,11,...,6.1,5.1,1.1,5.6,0.029,0.217,0.120,0.640,0.076,2019-20
11143,Norvel Pelle,PHI,27.0,208.28,104.779752,None,USA,Undrafted,Undrafted,Undrafted,...,2.1,3.0,0.4,-16.4,0.085,0.237,0.126,0.521,0.056,2019-20
11144,Matt Thomas,TOR,25.0,193.04,86.182480,Iowa State,USA,Undrafted,Undrafted,Undrafted,...,4.5,1.4,0.5,1.0,0.017,0.104,0.149,0.663,0.089,2019-20


Now we are on the court. We are playing a clean game with no fouls but... we need to enrich our stats. How? With information from an API. 

In [9]:
url = "https://nba-stats4.p.rapidapi.com/per_game_career_regular_season/"
query_string = {"per_page":"500","page":"1"}
headers = {
    'x-rapidapi-key': "d36955156cmsh88af60166373f21p15ef3fjsn367b0728f2e6",
    'x-rapidapi-host': "nba-stats4.p.rapidapi.com"
    }

In [10]:
response = requests.get(url, headers=headers, params = query_string).json()

In [12]:
players = [player["player_id"] for player in response]

In [13]:
players

[2,
 3,
 7,
 9,
 12,
 15,
 17,
 21,
 22,
 23,
 24,
 26,
 28,
 29,
 30,
 31,
 32,
 35,
 36,
 37,
 38,
 41,
 42,
 43,
 45,
 46,
 47,
 49,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 61,
 63,
 64,
 65,
 66,
 67,
 70,
 71,
 72,
 73,
 74,
 76,
 77,
 78,
 80,
 81,
 82,
 84,
 85,
 87,
 88,
 89,
 93,
 95,
 96,
 97,
 98,
 100,
 101,
 103,
 104,
 105,
 107,
 109,
 111,
 112,
 114,
 116,
 117,
 120,
 121,
 122,
 123,
 124,
 128,
 129,
 132,
 133,
 134,
 136,
 137,
 138,
 140,
 141,
 143,
 145,
 146,
 147,
 149,
 154,
 156,
 157,
 160,
 164,
 165,
 166,
 168,
 170,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 187,
 189,
 190,
 192,
 193,
 194,
 195,
 197,
 198,
 199,
 201,
 202,
 203,
 204,
 208,
 209,
 210,
 211,
 212,
 213,
 216,
 219,
 221,
 223,
 224,
 226,
 228,
 229,
 234,
 236,
 238,
 239,
 240,
 241,
 243,
 244,
 246,
 247,
 248,
 251,
 252,
 255,
 258,
 262,
 265,
 270,
 271,
 273,
 275,
 278,
 279,
 280,
 281,
 283,
 285,
 287,
 288,
 289,
 291,
 292,
 293,
 296

In [37]:
url_1 = "https://nba-stats4.p.rapidapi.com/players/"
query_string = {"per_page":"500","page":"1", "is_active" : "1"}
query_string_2 = {"per_page":"500","page":"2", "is_active" : "1"}
headers = {
    'x-rapidapi-key': "d36955156cmsh88af60166373f21p15ef3fjsn367b0728f2e6",
    'x-rapidapi-host': "nba-stats4.p.rapidapi.com"
    }

In [ ]:
response_players = requests.get(url_1, headers=headers, params = query_string).json()

In [41]:
response_players_2 = requests.get(url_1, headers=headers, params = query_string_2).json()

In [42]:
response_players

[{'first_name': 'Vince',
  'full_name': 'Vince Carter',
  'id': 1713,
  'is_active': 1,
  'last_name': 'Carter'},
 {'first_name': 'Tyson',
  'full_name': 'Tyson Chandler',
  'id': 2199,
  'is_active': 1,
  'last_name': 'Chandler'},
 {'first_name': None,
  'full_name': 'Nene',
  'id': 2403,
  'is_active': 1,
  'last_name': 'Nene'},
 {'first_name': 'LeBron',
  'full_name': 'LeBron James',
  'id': 2544,
  'is_active': 1,
  'last_name': 'James'},
 {'first_name': 'Carmelo',
  'full_name': 'Carmelo Anthony',
  'id': 2546,
  'is_active': 1,
  'last_name': 'Anthony'},
 {'first_name': 'Kyle',
  'full_name': 'Kyle Korver',
  'id': 2594,
  'is_active': 1,
  'last_name': 'Korver'},
 {'first_name': 'Udonis',
  'full_name': 'Udonis Haslem',
  'id': 2617,
  'is_active': 1,
  'last_name': 'Haslem'},
 {'first_name': 'Dwight',
  'full_name': 'Dwight Howard',
  'id': 2730,
  'is_active': 1,
  'last_name': 'Howard'},
 {'first_name': 'Andre',
  'full_name': 'Andre Iguodala',
  'id': 2738,
  'is_active': 1,

In [43]:
response_players_2

[{'first_name': 'John',
  'full_name': 'John Konchar',
  'id': 1629723,
  'is_active': 1,
  'last_name': 'Konchar'},
 {'first_name': 'Vic',
  'full_name': 'Vic Law',
  'id': 1629724,
  'is_active': 1,
  'last_name': 'Law'},
 {'first_name': 'Jeremiah',
  'full_name': 'Jeremiah Martin',
  'id': 1629725,
  'is_active': 1,
  'last_name': 'Martin'},
 {'first_name': 'Garrison',
  'full_name': 'Garrison Mathews',
  'id': 1629726,
  'is_active': 1,
  'last_name': 'Mathews'},
 {'first_name': 'Josh',
  'full_name': 'Josh Reaves',
  'id': 1629729,
  'is_active': 1,
  'last_name': 'Reaves'},
 {'first_name': 'Rayjon',
  'full_name': 'Rayjon Tucker',
  'id': 1629730,
  'is_active': 1,
  'last_name': 'Tucker'},
 {'first_name': 'Dean',
  'full_name': 'Dean Wade',
  'id': 1629731,
  'is_active': 1,
  'last_name': 'Wade'},
 {'first_name': 'Kyle',
  'full_name': 'Kyle Alexander',
  'id': 1629734,
  'is_active': 1,
  'last_name': 'Alexander'},
 {'first_name': 'Chris',
  'full_name': 'Chris Silva',
  'id':

In [44]:
list_full_name = []
list_player_id = []
full_names = [names["full_name"] for names in response_players]
list_full_name.append(full_names)
full_names_2 = [names["full_name"] for names in response_players_2]
list_full_name.append(full_names_2)
player_id = [ids["id"] for ids in response_players]
list_player_id.append(player_id)


In [45]:
list_full_name

[['Vince Carter',
  'Tyson Chandler',
  'Nene',
  'LeBron James',
  'Carmelo Anthony',
  'Kyle Korver',
  'Udonis Haslem',
  'Dwight Howard',
  'Andre Iguodala',
  'Trevor Ariza',
  'Marvin Williams',
  'Chris Paul',
  'Gerald Green',
  'Ian Mahinmi',
  'CJ Miles',
  'Ersan Ilyasova',
  'Lou Williams',
  'LaMarcus Aldridge',
  'Rudy Gay',
  'JJ Redick',
  'Thabo Sefolosha',
  'Rajon Rondo',
  'Kyle Lowry',
  'P.J. Tucker',
  'Paul Millsap',
  'J.J. Barea',
  'Kevin Durant',
  'Al Horford',
  'Mike Conley',
  'Jeff Green',
  'Thaddeus Young',
  'Marco Belinelli',
  'Jared Dudley',
  'Wilson Chandler',
  'Marc Gasol',
  'Anthony Tolliver',
  'Derrick Rose',
  'Russell Westbrook',
  'Kevin Love',
  'Danilo Gallinari',
  'Eric Gordon',
  'D.J. Augustin',
  'Brook Lopez',
  'Robin Lopez',
  'JaVale McGee',
  'Ryan Anderson',
  'Courtney Lee',
  'Serge Ibaka',
  'Nicolas Batum',
  'George Hill',
  'DeAndre Jordan',
  'Goran Dragic',
  'Blake Griffin',
  'James Harden',
  'Ricky Rubio',
  'St

In [34]:
list_player_id

[[1713,
  2199,
  2403,
  2544,
  2546,
  2594,
  2617,
  2730,
  2738,
  2772,
  101107,
  101108,
  101123,
  101133,
  101139,
  101141,
  101150,
  200746,
  200752,
  200755,
  200757,
  200765,
  200768,
  200782,
  200794,
  200826,
  201142,
  201143,
  201144,
  201145,
  201152,
  201158,
  201162,
  201163,
  201188,
  201229,
  201565,
  201566,
  201567,
  201568,
  201569,
  201571,
  201572,
  201577,
  201580,
  201583,
  201584,
  201586,
  201587,
  201588,
  201599,
  201609,
  201933,
  201935,
  201937,
  201939,
  201942,
  201949,
  201950,
  201952,
  201959,
  201960,
  201961,
  201976,
  201980,
  201988,
  202066,
  202083,
  202322,
  202323,
  202324,
  202326,
  202329,
  202330,
  202331,
  202334,
  202335,
  202339,
  202340,
  202355,
  202357,
  202397,
  202681,
  202683,
  202684,
  202685,
  202687,
  202688,
  202689,
  202691,
  202692,
  202693,
  202694,
  202695,
  202696,
  202697,
  202699,
  202704,
  202709,
  202710,
  202711,
  202718,


In [23]:
url_prueba = "https://nba-stats4.p.rapidapi.com/per_game_regular_season/"
query_string = {"per_page":"500","page":"1"}
headers = {
    'x-rapidapi-key': "d36955156cmsh88af60166373f21p15ef3fjsn367b0728f2e6",
    'x-rapidapi-host': "nba-stats4.p.rapidapi.com"
    }

In [24]:
list_season = []
list_full_name = []

In [25]:
for name in list_full_name:
    response_prueba = requests.get(url_prueba, headers = headers, params = query_string).json()
    list_season.append(response_prueba)



In [22]:
list_season

[]

In [ ]:
response_prueba = requests.get(url_prueba, headers = headers, params = query_string).json()
response_prueba

In [9]:
len(response_prueba)

500

In [10]:
season_2020 = response_prueba["data"]

TypeError: list indices must be integers or slices, not str

In [33]:
url_1 = "https://nba-stats4.p.rapidapi.com/per_game_regular_season/"
query_string = {"per_page":"500","page":"1"}
headers = {
    'x-rapidapi-key': "d36955156cmsh88af60166373f21p15ef3fjsn367b0728f2e6",
    'x-rapidapi-host': "nba-stats4.p.rapidapi.com"
    }

In [53]:
url = "https://free-nba.p.rapidapi.com/players"

querystring = {"page":"1","per_page":"100"}

headers = {
    'x-rapidapi-key': "d36955156cmsh88af60166373f21p15ef3fjsn367b0728f2e6",
    'x-rapidapi-host': "free-nba.p.rapidapi.com"
    }

response_4 = requests.get(url, headers=headers, params=querystring).json()


In [54]:
response_4["data"]

[{'first_name': 'Ike',
  'height_feet': None,
  'height_inches': None,
  'id': 14,
  'last_name': 'Anigbogu',
  'position': 'C',
  'team': {'abbreviation': 'IND',
   'city': 'Indiana',
   'conference': 'East',
   'division': 'Central',
   'full_name': 'Indiana Pacers',
   'id': 12,
   'name': 'Pacers'},
  'weight_pounds': None},
 {'first_name': 'Ron',
  'height_feet': None,
  'height_inches': None,
  'id': 25,
  'last_name': 'Baker',
  'position': 'G',
  'team': {'abbreviation': 'NYK',
   'city': 'New York',
   'conference': 'East',
   'division': 'Atlantic',
   'full_name': 'New York Knicks',
   'id': 20,
   'name': 'Knicks'},
  'weight_pounds': None},
 {'first_name': 'Jabari',
  'height_feet': None,
  'height_inches': None,
  'id': 47,
  'last_name': 'Bird',
  'position': 'G',
  'team': {'abbreviation': 'BOS',
   'city': 'Boston',
   'conference': 'East',
   'division': 'Atlantic',
   'full_name': 'Boston Celtics',
   'id': 2,
   'name': 'Celtics'},
  'weight_pounds': None},
 {'first

In [268]:
lista_nombres = []
lista_apellidos = []
lista_id = []

In [270]:
response = requests.get("https://www.balldontlie.io/api/v1/players?per_page=100&page=1").json()
names_pag_1 = response["data"]
names = [name["first_name"] for name in names_pag_1]
lista_nombres.append(names)
last_names = [surname["last_name"] for surname in names_pag_1]
lista_apellidos.append(last_names)
ids = [id_["id"] for id_ in names_pag_1]
lista_id.append(ids)

In [271]:
response = requests.get("https://www.balldontlie.io/api/v1/players?per_page=100&page=2").json()
names_pag_2 = response["data"]
names_2 = [name["first_name"] for name in names_pag_2]
lista_nombres.append(names_2)
last_names_2 = [surname["last_name"] for surname in names_pag_2]
lista_apellidos.append(last_names_2)
ids_2 = [id_["id"] for id_ in names_pag_2]
lista_id.append(ids_2)

In [272]:
response = requests.get("https://www.balldontlie.io/api/v1/players?per_page=100&page=3").json()
names_pag_3 = response["data"]
names_3 = [name["first_name"] for name in names_pag_3]
lista_jugadores.append(names_3)
last_names_3 = [surname["last_name"] for surname in names_pag_3]
lista_apellidos.append(last_names_3)
ids_3 = [id_["id"] for id_ in names_pag_3]
lista_id.append(ids_3)

In [273]:
response = requests.get("https://www.balldontlie.io/api/v1/players?per_page=100&page=4").json()
names_pag_4 = response["data"]
names_4 = [name["first_name"] for name in names_pag_4]
lista_jugadores.append(names_4)
last_names_4 = [surname["last_name"] for surname in names_pag_4]
lista_apellidos.append(last_names_4)
ids_4 = [id_["id"] for id_ in names_pag_4]
lista_id.append(ids_4)

In [274]:
response = requests.get("https://www.balldontlie.io/api/v1/players?per_page=100&page=5").json()
names_pag_5 = response["data"]
names_5 = [name["first_name"] for name in names_pag_5]
lista_jugadores.append(names_5)
last_names_5 = [surname["last_name"] for surname in names_pag_5]
lista_apellidos.append(last_names)
ids_5 = [id_["id"] for id_ in names_pag_5]
lista_id.append(ids_5)

In [275]:
response = requests.get("https://www.balldontlie.io/api/v1/players?per_page=100&page=10").json()

In [276]:
response["data"]

[{'first_name': 'Jason',
  'height_feet': None,
  'height_inches': None,
  'id': 1372,
  'last_name': 'Richardson',
  'position': '',
  'team': {'abbreviation': 'GSW',
   'city': 'Golden State',
   'conference': 'West',
   'division': 'Pacific',
   'full_name': 'Golden State Warriors',
   'id': 10,
   'name': 'Warriors'},
  'weight_pounds': None},
 {'first_name': 'Troy',
  'height_feet': None,
  'height_inches': None,
  'id': 1373,
  'last_name': 'Murphy',
  'position': '',
  'team': {'abbreviation': 'GSW',
   'city': 'Golden State',
   'conference': 'West',
   'division': 'Pacific',
   'full_name': 'Golden State Warriors',
   'id': 10,
   'name': 'Warriors'},
  'weight_pounds': None},
 {'first_name': 'Gilbert',
  'height_feet': None,
  'height_inches': None,
  'id': 1374,
  'last_name': 'Arenas',
  'position': '',
  'team': {'abbreviation': 'GSW',
   'city': 'Golden State',
   'conference': 'West',
   'division': 'Pacific',
   'full_name': 'Golden State Warriors',
   'id': 10,
   'nam

In [12]:
#STOPPPPPPP START WEB SCRAPING